In [1]:
import numpy as np
import pandas as pd
import datetime
import copy
import time
import os
import re
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import operator

from tqdm.auto import tqdm, trange
from tqdm.notebook import tqdm
from datetime import timedelta

tqdm.pandas()

In [2]:
# Edit to point to your MIMIC directory.
dataDirStr = '/Users/gmessier/data/mimic-1.4/'

In [3]:
d_labitems_df = pd.read_csv(dataDirStr + "D_LABITEMS.csv")
d_labitems_df.columns = d_labitems_df.columns.str.lower()
d_labitems_df

,row_id,itemid,label,fluid,category,loinc_code
0,546,51346,Blasts,Cerebrospinal Fluid (CSF),Hematology,26447-3
1,547,51347,Eosinophils,Cerebrospinal Fluid (CSF),Hematology,26451-5
2,548,51348,"Hematocrit, CSF",Cerebrospinal Fluid (CSF),Hematology,30398-2
3,549,51349,Hypersegmented Neutrophils,Cerebrospinal Fluid (CSF),Hematology,26506-6
4,550,51350,Immunophenotyping,Cerebrospinal Fluid (CSF),Hematology,NaN
...,...,...,...,...,...,...
748,749,51551,VOIDED SPECIMEN,OTHER BODY FLUID,HEMATOLOGY,NaN
749,750,51552,VOIDED SPECIMEN,STOOL,CHEMISTRY,NaN
750,751,51553,VOIDED SPECIMEN,URINE,CHEMISTRY,NaN
751,752,51554,VOIDED SPECIMEN,JOINT FLUID,HEMATOLOGY,NaN


`D_LABITEMS` contains definitions for all `itemid` associated with lab measurements in the MIMIC database. All data in `LABEVENTS` link to the `D_LABITEMS` table.

Most concepts in this table have been mapped to `LOINC` codes, an openly available ontology which provides a rich amount of information about the laboratory measurement including reference ranges, common units of measurement and other further detail regarding the measurement.

The `label` column describes the concept which is represented by the `itemid`.

In [4]:
d_labitems_df.label.value_counts()[:5]

VOIDED SPECIMEN    12
Eosinophils         7
Bands               6
Basophils           6
Metamyelocytes      6
Name: label, dtype: int64

`fluid` describes the substance on which the measurement was made. For example, chemistry measurements are frequently performed on blood, which is listed in this column as ‘BLOOD’. Many of these measurements are also acquirable on other fluids, such as urine, and this column differentiates these distinct concepts.



In [5]:
d_labitems_df.fluid.value_counts()[:5]

Blood               359
Urine               103
Other Body Fluid     82
Bone Marrow          40
Ascites              37
Name: fluid, dtype: int64

`category` provides higher level information as to the type of measurement. For example, a category of ‘ABG’ indicates that the measurement is an arterial blood gas.

In [6]:
d_labitems_df.category.value_counts()[:5]

Hematology    410
Chemistry     274
Blood Gas      34
CHEMISTRY      19
HEMATOLOGY     13
Name: category, dtype: int64

`LOINC_CODE` contains the `LOINC` code associated with the given `itemid`.  `LOINC` is an ontology which originally specified laboratory measurements but has since expanded to cover a wide range of clinically relevant concepts. `LOINC` openly provide a table which contains a large amount of detail about each `LOINC` code. This table is freely available online.